<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Clima/SMN_DIARIOS_JAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
import csv
import time
import re
import os
from datetime import datetime
import pandas as pd

# Configuración para Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
except:
    IN_COLAB = False

# Configuración
BASE_URL = "https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Diarios/jal"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9',
}

# Variables disponibles en los datos diarios con nuevos nombres
VARIABLES_DIARIAS = ['p', 'evap', 'tmax', 'tmin']
VARIABLES_ORIGINALES = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']
VARIABLE_MAPPING = dict(zip(VARIABLES_ORIGINALES, VARIABLES_DIARIAS))

# Valores que deben ser reemplazados por NA (incluyendo NULO)
VALORES_NULOS = ['', 'N/D', '-', '--', '---', '////', '****', '*****', 'NULL', 'null', 'N/A', 'n/a', 'NULO']

def configurar_entorno_colab():
    """Configura el entorno para Google Colab"""
    if IN_COLAB:
        print("Montando Google Drive...")
        drive.mount('/content/drive')

        # Crear directorio base si no existe
        directorio_base = "/content/drive/MyDrive/Clima_Jalisco"
        directorio_datos = "/content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS"

        os.makedirs(directorio_base, exist_ok=True)
        os.makedirs(directorio_datos, exist_ok=True)

        print(f"Directorio de trabajo configurado en: {directorio_datos}")
        return directorio_datos
    else:
        # Si no está en Colab, usar directorio local
        directorio_local = "Clima_Jalisco/DATOS_DIARIOS"
        os.makedirs(directorio_local, exist_ok=True)
        return directorio_local

def descargar_datos_estacion(clave):
    """Descarga los datos diarios de una estación específica"""
    url = f"{BASE_URL}/dia{clave}.txt"
    try:
        print(f"  Descargando de: {url}")
        response = requests.get(url, headers=HEADERS, timeout=30)
        response.raise_for_status()

        # Verificar si el contenido es válido
        if len(response.text.strip()) < 50:
            print(f"  Contenido insuficiente para estación {clave}")
            return None

        return response.text
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"  Estación {clave} no encontrada (404)")
        else:
            print(f"  Error HTTP {e.response.status_code} para estación {clave}")
        return None
    except Exception as e:
        print(f"  Error descargando datos para estación {clave}: {str(e)}")
        return None

def extraer_info_basica(contenido):
    """Extrae la información básica de la estación"""
    datos = {
        'Codigo': '',
        'site': '',
        'estado': 'Jalisco',
        'municipio': '',
        'situacion': '',
        'lat': '',
        'lon': '',
        'alt': ''
    }

    lineas = contenido.split('\n')
    for linea in lineas:
        linea = linea.strip()
        if 'ESTACIÓN  :' in linea:
            datos['Codigo'] = linea.split(':')[1].strip()
        elif 'NOMBRE    :' in linea:
            datos['site'] = linea.split(':')[1].strip()
        elif 'MUNICIPIO :' in linea:
            datos['municipio'] = linea.split(':')[1].strip()
        elif 'SITUACIÓN :' in linea:
            datos['situacion'] = linea.split(':')[1].strip()
        elif 'LATITUD   :' in linea:
            datos['lat'] = linea.split(':')[1].replace('°', '').strip()
        elif 'LONGITUD  :' in linea:
            datos['lon'] = linea.split(':')[1].replace('°', '').strip()
        elif 'ALTITUD   :' in linea:
            datos['alt'] = linea.split(':')[1].replace('msnm', '').strip()

    return datos

def limpiar_valor(valor):
    """Limpia y valida los valores, reemplazando nulos por NA"""
    if valor is None:
        return 'NA'

    valor_str = str(valor).strip()

    # Si está vacío o en la lista de valores nulos (incluyendo NULO)
    if not valor_str or valor_str in VALORES_NULOS:
        return 'NA'

    # Si son solo asteriscos o caracteres no válidos
    if re.match(r'^[*\-/\s]+$', valor_str):
        return 'NA'

    return valor_str

def parsear_datos_diarios(contenido):
    """Extrae los datos diarios del contenido, manejando valores NULO"""
    lineas = contenido.split('\n')
    datos_diarios = []

    # Bandera para indicar cuando empezar a leer datos
    empezar_datos = False
    patron_fecha_guion = re.compile(r'^\d{4}-\d{1,2}-\d{1,2}')  # Formato YYYY-MM-DD
    patron_fecha_slash = re.compile(r'^\d{1,2}/\d{1,2}/\d{4}')  # Formato DD/MM/YYYY

    # Buscar la línea de encabezados
    for i, linea in enumerate(lineas):
        linea = linea.strip()

        # Buscar el inicio de la tabla de datos
        if 'FECHA' in linea and any(var in linea for var in VARIABLES_ORIGINALES):
            empezar_datos = True
            # Saltar la siguiente línea (unidades) si existe
            continue

        # Si estamos en la sección de datos, procesar línea
        if empezar_datos and linea:
            # Verificar si es una línea de datos (comienza con fecha en formato YYYY-MM-DD o DD/MM/YYYY)
            if patron_fecha_guion.match(linea) or patron_fecha_slash.match(linea):
                # Dividir por tabs (parece que están separados por tabs en el ejemplo)
                partes = linea.split('\t')

                # Si no hay tabs, intentar con espacios múltiples
                if len(partes) < 2:
                    partes = re.split(r'\s{2,}', linea)

                # Si aún no funciona, usar split simple
                if len(partes) < 2:
                    partes = linea.split()

                # Filtrar partes vacías
                partes = [p.strip() for p in partes if p.strip()]

                # Debe tener al menos la fecha
                if len(partes) >= 1:
                    fecha = partes[0]

                    # Procesar valores para cada variable
                    valores = {}

                    # Asignar valores según posición esperada
                    # Orden esperado: FECHA, PRECIP, EVAP, TMAX, TMIN
                    if len(partes) > 1:
                        valores['p'] = limpiar_valor(partes[1])
                    else:
                        valores['p'] = 'NA'

                    if len(partes) > 2:
                        valores['evap'] = limpiar_valor(partes[2])
                    else:
                        valores['evap'] = 'NA'

                    if len(partes) > 3:
                        valores['tmax'] = limpiar_valor(partes[3])
                    else:
                        valores['tmax'] = 'NA'

                    if len(partes) > 4:
                        valores['tmin'] = limpiar_valor(partes[4])
                    else:
                        valores['tmin'] = 'NA'

                    # Crear registro
                    registro = {
                        'date': fecha,
                        **valores
                    }
                    datos_diarios.append(registro)

            # Si encontramos líneas que indican el final de los datos
            elif any(palabra in linea.upper() for palabra in ['PROMEDIO', 'TOTAL', 'MEDIA', 'FIN DE DATOS']):
                break

    # Si no encontramos datos con el método anterior, intentar método más flexible
    if not datos_diarios:
        datos_diarios = parsear_datos_flexible(contenido)

    print(f"    Se extrajeron {len(datos_diarios)} registros diarios")

    # Depuración: mostrar primeros 3 registros si hay datos
    if datos_diarios:
        print(f"    Primeros registros: {datos_diarios[:3]}")

    return datos_diarios

def parsear_datos_flexible(contenido):
    """Método más flexible para extraer datos cuando el método principal falla"""
    lineas = contenido.split('\n')
    datos_diarios = []

    # Patrones de fecha
    patron_fecha_guion = re.compile(r'^\d{4}-\d{1,2}-\d{1,2}')  # YYYY-MM-DD
    patron_fecha_slash = re.compile(r'^\d{1,2}/\d{1,2}/\d{4}')   # DD/MM/YYYY

    # Buscar cualquier línea que empiece con fecha
    for linea in lineas:
        linea = linea.strip()

        # Verificar si es una línea de datos (comienza con fecha)
        if patron_fecha_guion.match(linea) or patron_fecha_slash.match(linea):
            # Dividir por cualquier tipo de espacio (tabs, espacios múltiples)
            partes = re.split(r'\t|\s{2,}', linea)

            # Si no funciona, usar split simple
            if len(partes) < 2:
                partes = linea.split()

            # Filtrar partes vacías
            partes = [p.strip() for p in partes if p.strip()]

            if len(partes) >= 1:
                fecha = partes[0]
                valores = {}

                # Mapear las variables por posición
                for i, var in enumerate(VARIABLES_DIARIAS):
                    if i + 1 < len(partes):
                        valores[var] = limpiar_valor(partes[i + 1])
                    else:
                        valores[var] = 'NA'

                registro = {'date': fecha, **valores}
                datos_diarios.append(registro)

    return datos_diarios

def guardar_datos_estacion(directorio_salida, estacion_info, datos_diarios):
    """Guarda los datos diarios de una estación en archivo CSV"""
    if not datos_diarios:
        print(f"  No hay datos diarios para guardar de la estación {estacion_info['Codigo']}")
        return False

    # Crear nombre de archivo
    archivo_csv = os.path.join(directorio_salida, f"diarios_{estacion_info['Codigo']}.csv")

    # Campos del CSV con nuevos nombres
    campos = ['Codigo', 'site', 'municipio', 'lat', 'lon', 'alt', 'date'] + VARIABLES_DIARIAS

    try:
        # Escribir datos
        with open(archivo_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=campos)
            writer.writeheader()

            for registro in datos_diarios:
                fila = {
                    'Codigo': estacion_info['Codigo'],
                    'site': estacion_info['site'],
                    'municipio': estacion_info['municipio'],
                    'lat': estacion_info['lat'],
                    'lon': estacion_info['lon'],
                    'alt': estacion_info['alt'],
                    'date': registro['date'],
                    'p': registro.get('p', 'NA'),
                    'evap': registro.get('evap', 'NA'),
                    'tmax': registro.get('tmax', 'NA'),
                    'tmin': registro.get('tmin', 'NA')
                }
                writer.writerow(fila)

        print(f"  Datos guardados en: {archivo_csv}")
        return True
    except Exception as e:
        print(f"  Error guardando archivo para estación {estacion_info['Codigo']}: {str(e)}")
        return False

def guardar_datos_unidos(directorio_salida, estacion_info, datos_diarios):
    """Guarda todos los datos en un archivo CSV unificado"""
    if not datos_diarios:
        return

    archivo_unido = os.path.join(directorio_salida, "datos_diarios_unidos.csv")

    # Campos del CSV con nuevos nombres
    campos = ['Codigo', 'site', 'municipio', 'lat', 'lon', 'alt', 'date'] + VARIABLES_DIARIAS

    try:
        # Verificar si el archivo existe para no sobrescribir el encabezado
        modo = 'a' if os.path.exists(archivo_unido) else 'w'

        with open(archivo_unido, modo, newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=campos)

            if modo == 'w':
                writer.writeheader()
                print(f"  Creando archivo unificado: {archivo_unido}")

            for registro in datos_diarios:
                fila = {
                    'Codigo': estacion_info['Codigo'],
                    'site': estacion_info['site'],
                    'municipio': estacion_info['municipio'],
                    'lat': estacion_info['lat'],
                    'lon': estacion_info['lon'],
                    'alt': estacion_info['alt'],
                    'date': registro['date'],
                    'p': registro.get('p', 'NA'),
                    'evap': registro.get('evap', 'NA'),
                    'tmax': registro.get('tmax', 'NA'),
                    'tmin': registro.get('tmin', 'NA')
                }
                writer.writerow(fila)
    except Exception as e:
        print(f"  Error actualizando archivo unificado: {str(e)}")

def procesar_estacion(clave, directorio_salida):
    """Procesa una estación individual"""
    print(f"Procesando estación: {clave}")

    contenido = descargar_datos_estacion(clave)

    if not contenido:
        print(f"  No se pudieron obtener datos para la estación {clave}")
        return

    # Extraer información básica de la estación
    estacion_info = extraer_info_basica(contenido)

    # Si no se pudo extraer la clave, usar la clave proporcionada
    if not estacion_info['Codigo']:
        estacion_info['Codigo'] = clave

    print(f"  Estación: {estacion_info['site']} ({estacion_info['municipio']})")

    # Extraer datos diarios
    datos_diarios = parsear_datos_diarios(contenido)

    if datos_diarios:
        # Guardar en archivo individual de la estación
        guardar_datos_estacion(directorio_salida, estacion_info, datos_diarios)

        # Guardar en archivo unificado
        guardar_datos_unidos(directorio_salida, estacion_info, datos_diarios)
    else:
        print(f"  No se encontraron datos diarios para esta estación")
        # Depuración: mostrar algunas líneas del contenido para diagnóstico
        lineas = contenido.split('\n')
        print(f"  Primeras 15 líneas del contenido para diagnóstico:")
        for i, linea in enumerate(lineas[:15]):
            print(f"    Línea {i}: {repr(linea)}")

    time.sleep(1)  # Espera entre descargas para no saturar el servidor

def leer_archivo_estaciones(nombre_archivo):
    """Lee el archivo de estaciones con manejo de codificaciones"""
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as f:
            lineas = [linea.strip() for linea in f.readlines() if linea.strip()]
            print(f"Se leyeron {len(lineas)} estaciones del archivo")
            return lineas
    except UnicodeDecodeError:
        try:
            with open(nombre_entrada, 'r', encoding='latin-1') as f:
                lineas = [linea.strip() for linea in f.readlines() if linea.strip()]
                print(f"Se leyeron {len(lineas)} estaciones del archivo (codificación latin-1)")
                return lineas
        except Exception as e:
            print(f"Error leyendo archivo de estaciones: {str(e)}")
            return []
    except Exception as e:
        print(f"Error leyendo archivo de estaciones: {str(e)}")
        return []

def generar_reporte_estaciones(directorio_salida):
    """Genera un reporte de las estaciones procesadas"""
    archivos = [f for f in os.listdir(directorio_salida) if f.startswith('diarios_') and f.endswith('.csv')]

    reporte = []
    for archivo in archivos:
        clave = archivo.replace('diarios_', '').replace('.csv', '')
        ruta_archivo = os.path.join(directorio_salida, archivo)

        try:
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                lineas = f.readlines()
                if len(lineas) > 1:  # Tiene encabezado y al menos un dato
                    num_registros = len(lineas) - 1
                    reporte.append({
                        'Codigo': clave,
                        'archivo': archivo,
                        'registros': num_registros
                    })
        except:
            continue

    # Guardar reporte
    if reporte:
        archivo_reporte = os.path.join(directorio_salida, "reporte_estaciones.csv")
        with open(archivo_reporte, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['Codigo', 'archivo', 'registros'])
            writer.writeheader()
            writer.writerows(reporte)

        print(f"\nReporte generado: {archivo_reporte}")
        print(f"Total de estaciones con datos: {len(reporte)}")

        # Mostrar resumen
        for est in reporte:
            print(f"  {est['Codigo']}: {est['registros']} registros")

def main():
    print("Sistema de Extracción de Datos Meteorológicos Diarios de Jalisco")
    print("Variables: date, p, evap, tmax, tmin")
    print("Valores nulos (incluyendo 'NULO') se reemplazan por: NA")

    # Configurar entorno (Colab o local)
    directorio_salida = configurar_entorno_colab()

    # Opción 1: Procesar desde archivo de entrada
    archivo_entrada = "estaciones.txt"

    # Si estamos en Colab, buscar el archivo en Drive también
    if IN_COLAB and not os.path.exists(archivo_entrada):
        # Intentar encontrar el archivo en diferentes ubicaciones de Drive
        posibles_rutas = [
            "/content/drive/MyDrive/estaciones.txt",
            "/content/drive/MyDrive/Clima_Jalisco/estaciones.txt",
            "/content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS/estaciones.txt",
            "/content/estaciones.txt"
        ]

        for ruta in posibles_rutas:
            if os.path.exists(ruta):
                archivo_entrada = ruta
                print(f"Archivo de estaciones encontrado en: {ruta}")
                break

    estaciones = []

    if os.path.exists(archivo_entrada):
        estaciones = leer_archivo_estaciones(archivo_entrada)
    else:
        print(f"Archivo {archivo_entrada} no encontrado")

    # Si no hay estaciones del archivo, usar lista por defecto
    if not estaciones:
        print("Usando lista de estaciones por defecto")
        estaciones = ['14001', '14002', '14003', '14004', '14005',
                     '14006', '14007', '14008', '14009', '14010',
                     '14011', '14012', '14013', '14014', '14015']

    if estaciones:
        total = len(estaciones)
        print(f"\nProcesando {total} estaciones...")

        exitosas = 0
        for i, clave in enumerate(estaciones, 1):
            print(f"\n[{i}/{total}] ", end="")
            procesar_estacion(clave, directorio_salida)
            if os.path.exists(os.path.join(directorio_salida, f"diarios_{clave}.csv")):
                exitosas += 1

        print(f"\n¡Proceso completado!")
        print(f"Estaciones con datos extraídos: {exitosas}/{total}")
        print(f"Datos guardados en: {directorio_salida}")

        # Generar reporte
        generar_reporte_estaciones(directorio_salida)
    else:
        print("No se encontraron estaciones para procesar")

if __name__ == "__main__":
    main()

Sistema de Extracción de Datos Meteorológicos Diarios de Jalisco
Variables: date, p, evap, tmax, tmin
Valores nulos (incluyendo 'NULO') se reemplazan por: NA
Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo configurado en: /content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS
Archivo de estaciones encontrado en: /content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS/estaciones.txt
Se leyeron 275 estaciones del archivo

Procesando 275 estaciones...

[1/275] Procesando estación: 14001
  Descargando de: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Diarios/jal/dia14001.txt
  Estación: ACATIC (SMN) (ACATIC)
    Se extrajeron 4429 registros diarios
    Primeros registros: [{'date': '1961-01-01', 'p': '0', 'evap': 'NA', 'tmax': 'NA', 'tmin': 'NA'}, {'date': '1961-01-02', 'p': '0', 'evap': 'NA', 'tmax': 'NA', 'tmin': 'NA'}, {'date': '1961-01-03', 'p': '0